In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import h2o
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from h2o.estimators.gbm import H2OGradientBoostingEstimator

h2o.init()



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment AdoptOpenJDK (build 11.0.4+11); OpenJDK 64-Bit Server VM AdoptOpenJDK (build 11.0.4+11, mixed mode)
  Starting server from /opt/anaconda3/envs/h2oprod-py3.8/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/5j/hpc15scd6nx23pl4x9tzr2mw0000gp/T/tmp648hjimy
  JVM stdout: /var/folders/5j/hpc15scd6nx23pl4x9tzr2mw0000gp/T/tmp648hjimy/h2o_gerd_started_from_python.out
  JVM stderr: /var/folders/5j/hpc15scd6nx23pl4x9tzr2mw0000gp/T/tmp648hjimy/h2o_gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Vienna
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_gerd_p5ad06
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [2]:

# In[2]:


N = 1000
bloodTypes = ['A', 'O', 'AB', 'B']
np.random.seed(1)

id = range(0,N)
bloodTypesSample = [bloodTypes[z] for z in np.random.randint(0, 4, N)]
age = np.random.randint(18, 65, N)
healthyEating = np.round(np.random.normal(5, 2, N),0)
activeLifestyle = np.round(np.random.normal(5, 2, N),0)

for i in range(0,N):
    if healthyEating[i] > 9.:
        healthyEating[i] = 9
    if healthyEating[i] < 0.:
        healthyEating[i] = 0

for i in range(0,N):
    if age[i] < 30:
        activeLifestyle[i]+=1
    if activeLifestyle[i] > 9.:
        activeLifestyle[i] = 9
    if activeLifestyle[i] < 0.:
        activeLifestyle[i] = 0

income = [round(20000 + (age[i]*3)**2 + healthyEating[i] * 500 - activeLifestyle[i] * 300 + np.random.randint(0, 5000), -2)
             for i in range(0,N)]



In [3]:

# In[3]:


people = pd.DataFrame({
    'Id':id,
    'Blood Type':bloodTypesSample, 
    'Age':age, 
    'Healthy Eating':healthyEating, 
    'Active Lifestyle':activeLifestyle, 
    'Income':income
})

people = h2o.H2OFrame(
    people
    )



Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:

# In[4]:


people.describe()


# In[5]:


## Training to first model

train, valid, test = people.split_frame(
    ratios = [0.8, 0.1],
    destination_frames=['people_train','people_valid','people_test'],
    seed = 123
)

y = 'Income'
ignoreFields = [y, 'Id']
x = [i for i in train.names if i not in ignoreFields]

ml = H2OGradientBoostingEstimator(model_id = 'defaults')
ml.train(x, y, train, validation_frame = valid)


print('Train sample MAE:', ml.mae(train=True))
print('Validation sample MAE:', ml.mae(valid=True))

perf = ml.model_performance(test)

print('Perfomance MAE:', perf.mae()) 



Rows:1000
Cols:6




,Id,Blood Type,Age,Healthy Eating,Active Lifestyle,Income
type,int,enum,int,int,int,int
mins,0.0,,18.0,0.0,0.0,22500.0
mean,499.5,,40.846000000000075,4.983999999999992,5.313000000000001,39972.09999999998
maxs,999.0,,64.0,9.0,9.0,64600.0
sigma,288.8194360957494,,13.420670564409152,2.0317164841266284,1.9382434800065216,10193.543049887505
zeros,1,,0,16,8,0
missing,0,0,0,0,0,0
0,0.0,O,22.0,5.0,5.0,30100.0
1,1.0,B,27.0,8.0,4.0,31300.0
2,2.0,A,18.0,6.0,5.0,27900.0


gbm Model Build progress: |███████████████████████████████████████████████| 100%
Train sample MAE: 997.2082730210977
Validation sample MAE: 1229.7792052372718
Perfomance MAE: 1254.4055872469512


In [13]:

# In[6]:


## Training the second model (cross-validation)

train_cv, test_cv = people.split_frame(
    ratios = [0.85],
    destination_frames=['people_train_cv','people_test_cv'],
    seed = 123
)


ml_cv = H2OGradientBoostingEstimator(model_id = 'def9folds', ntrees=150, nfolds = 8, max_depth=5, learn_rate=0.1)
ml_cv.train(x, y, train)

print('Train sample MAE:', ml_cv.mae(train=True))
print('Validation sample MAE:', ml_cv.mae(xval=True))

perf_cv = ml_cv.model_performance(test)
print('Perfomance MAE:', perf_cv.mae()) 



gbm Model Build progress: |███████████████████████████████████████████████| 100%
Train sample MAE: 894.9269288348668
Validation sample MAE: 1374.0514820377552
Perfomance MAE: 1316.0259646207978
